In [ ]:
def get_spectr(np_wav_files, nperseg=512, samplerate=24000):
    output = list()
    for i, wav in enumerate(np_wav_files):
        _, _, spect = signal.stft(wav, samplerate, nperseg=nperseg)
        output.append(spect)
        if i % 500 == 0: print(i)
    return output

In [ ]:
spect_wav = get_spectr(np_wav_files)

In [ ]:
def get_pkl(file_path):
    with open(file_path, 'rb') as file:
        loaded_list = pickle.load(file)
    return loaded_list

In [1]:
print(1)

1
